# Cleaning



<img src="https://media.tenor.com/oNn3Xf14XrkAAAAC/consuela-family-guy.gif">

Columns are the same for each dataset.     
So we can write one script to clean them all. (Test on one for loop on others)

### Imports and setup

In [6]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
import seaborn as sns
import sys
import demoji
sys.path.append('../')



pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', 250)



data_path = os.path.join('Data','finance','submissions_reddit.csv')


### Before data can be read in to dataframe, I think it is necessary to do some preprocessing to the csv file itself

In [7]:
df = pd.read_csv(data_path)

In [11]:
df.head(1)

,id,author,created,retrieved,edited,pinned,archived,locked,removed,deleted,is_self,is_video,is_original_content,title,link_flair_text,upvote_ratio,score,gilded,total_awards_received,num_comments,num_crossposts,selftext,thumbnail,shortlink
0,ko3ecd,bighlovd,2021-01-01 02:33:23,2021-02-04 06:28:30,1970-01-01 00:00:00,0,0,0,1,0,0,0,0,click this link to make free sign up!!!,None,0.5,0,0,0,0,0,NaN,default,https://redd.it/ko3ecd


Columns:    
| Index | Feature               | Type     | Description                                                    |
|-------|-----------------------|----------|----------------------------------------------------------------|
| 0     | id                    | string   | The id of the submission                                       |
| 1     | author                | string   | The redditors username                                         |
| 2     | created               | datetime | Time the submission was created                                |
| 3     | retrieved             | datetime | Time the submission was retrieved                              |
| 4     | edited                | datetime | Time the submission was modified                               |
| 5     | pinned                | boolean  | Whether or not the submission is pinned                        |
| 6     | archived              | boolean  | Whether or not the submission is archived                      |
| 7     | locked                | boolean  | Whether or not the submission is locked                        |
| 8     | removed               | boolean  | Whether or not the submission is removed                       |
| 9     | deleted               | boolean  | Whether or not the submission is user deleted                  |
| 10    | is_self               | boolean  | Whether or not the submission is a text                        |
| 11    | is_video              | boolean  | Whether or not the submission is a video                       |
| 12    | is_original_content   | boolean  | Whether or not the submission has been set as original content |
| 13    | title                 | string   | Title of the submission                                        |
| 14    | link_flair_text       | string   | Submission link flairs text content                            |
| 15    | upvote_ratio          | double   | Percentage of upvotes from all votes on submission             |
| 16    | score                 | integer  | number of upvotes                                              |
| 17    | gilded                | integer  | number of gilded awards                                        |
| 18    | total_awards_received | integer  | number of awards on the submission                             |
| 19    | num_comments          | integer  | number of comments on the submission                           |
| 20    | num_crossposts        | integer  | number of crossposts on the submission                         |
| 21    | selftext              | string   | submission selftext on text posts                              |
| 22    | thumbnail             | string   | submission thumbnail on image posts                            |
| 23    | shortlink             | string   | submission short url                                           |    

### Cleaning functions for Title


Things that need to be cleaned from "Title":
- New lines
- Emojis (convert or remove?)
- Spam messages (possibly only take posts that have a certain number of upvotes)
- links (need to remove entire record if link is only thing)
- videos (same as link)
- A lot of records do not talk about a specific stock. (Remove them?)

In [3]:
def remove_spacing(x):
    x.replace('\n', '')
    

def remove_videos(x):
    pass

def remove_irrelevant_posts(x):
    pass

def convert_emojis(x):
    demoji.replace_with_desc(x, sep='"')
    


In [4]:
df['title'][694]

'Highest percentage increase in share amount of different ARK ETFs - 5 Feb 2021'

Columns that can be removed for sure:
- id
- shortlink  
- thumbnail


Columns that could maybe possibly be removed:
- retrieved
- edited 
- pinned  
- archived  
- locked  
- removed (if removed is true should we discard the record?)
- deleted (same as removed)
- is_self   
- is_video (use as flag to remove records?)
- gilded   
...